In [1]:
import json
import dlite
import os
import pandas as pd
from bokeh_apps import InstanceApp

In [2]:
THISDIR = os.path.abspath('')
PATH_DATAMODELS = os.path.join(THISDIR, f'../entities/', "BatteryArchiveTimeSeriesData.json")
URLS_CSV = "https://www.batteryarchive.org/data/CALCE_CX2-16_prism_LCO_25C_0-100_0.5-0.5C_a_timeseries.csv"

In [4]:
BatteryTimeSeriesData = dlite.Instance.from_url(f"json://{PATH_DATAMODELS}")

In [5]:
def print_dlite_instance(dlite_instance):

    for key, value in dlite_instance.asdict().items():

        print(key)
        if isinstance(value, str):
            print(value, "\n")
        elif isinstance(value, dict):
            print(list(value.keys()), "\n")

In [3]:
df = pd.read_csv(URLS_CSV)
df.columns

Index(['Date_Time', 'Test_Time (s)', 'Cycle_Index', 'Current (A)',
       'Voltage (V)', 'Charge_Capacity (Ah)', 'Discharge_Capacity (Ah)',
       'Charge_Energy (Wh)', 'Discharge_Energy (Wh)',
       'Environment_Temperature (C)', 'Cell_Temperature (C)'],
      dtype='object')

In [6]:
df.shape

(820996, 11)

In [ ]:
conformed_data = BatteryTimeSeriesData(dims=[10])

: 

: 

In [11]:
#Use datamodel object to create a new Dlite instance: data conformed to the datamodel
conformed_data = BatteryTimeSeriesData(dims=[df.shape[0]])

for column in df.columns

#populed conformed data object
conformed_data.battery_id = raw_data.rows*[raw_data.meta.asdict()["uri"].split("/")[-1]]
conformed_data.test_time = raw_data.time
conformed_data.battery_voltage = raw_data.voltage
conformed_data.battery_current = raw_data.current
conformed_data.battery_temperature = raw_data.cell_temperature
conformed_data.environment_voltage = raw_data.environment_temperature


print_dlite_instance(conformed_data)

uuid
50d2ca2f-dd3d-4c35-863c-3fd0ea36a73c 

meta
http://onto-ns.com/meta/0.1/BatteryTimeSeriesData 

dimensions
['n_measurements'] 

properties
['battery_id', 'date_time_stamp', 'test_time', 'battery_current', 'battery_voltage', 'battery_temperature', 'battery_temperature_location', 'environment_temperature', 'environment_temperature_source'] 



# Data conversions

What type of information is needed to convert a stored file into usable data for an application?  

1. Load  

    1.1 where the file is  
      
    1.2 what is its format (json, csv, ...)  


2. Convert  

    2.1 Template of new datamodel  

    2.2 Function to populate template of new datamodel  

    2.3 Template of old datamodel  

    2.4 Function to access data with old datamodel  

    2.5 An intermediate, universally-accessible object  
    

In [9]:
def load_data_from_json(path_dataset:str, path_datamodel:str = ""):
    """
    Loads a dataset in json format into memory using a datamodel as template.

    Parameters:
        path_dataset: string
            path where the dataset is stored
        path_datamodel_in: string
            path where datamodel is stored
    Returns:
        out: object
            object with the attributes specified by path_datamodel
    """

    with open(path_dataset) as f:
        data_instance = json.load(f)

    return data_instance




def convert_data(data_instance, datamodel_path: str = ""):
    """
    Trasnform a data instance into a new format, using a datamodel as template.

    Parameters:
        data_instance: object
            data instance to be transformed to new format
        path_datamodel_in: string
            path where datamodel is stored
    Returns:
        out: object
            object with the attributes specified by datamodel
    """
    #template of new datamodel: a dict of (col_name, array) pairs. 
    # At least one column 'id' of indexes/strings used to slice the columns with data
    data_new_format = {} 

    #function accessing dataset. Made by visual inspection of the json file
    for instance_label, instance_data in data_instance.items(): 

        if "n_measurements" not in instance_data["dimensions"].keys():
            continue

        n_measurements = instance_data["dimensions"]["n_measurements"]

        if not data_new_format:
            data_new_format.update(instance_data["properties"])
            data_new_format["id"] = n_measurements*[instance_label]
        else:
            for key, value in instance_data["properties"].items():
                data_new_format[key].extend(value)
            data_new_format["id"].extend(n_measurements*[instance_label])
    
    return data_new_format

## Bokeh self standing app
Follow examples from the [blog post](http://justinbois.github.io/bootcamp/2021/lessons/l30_javascript_for_bokeh.html)

In [10]:
data_instance = load_data_from_json(path_dataset="rwth_cycledata_collection_processed.json")
data_new_format = convert_data(data_instance = data_instance)
app = AppTimeSeries(data_new_format)
app.render_app()
# app.save_app("app_rwth_cycledata_collection_processed.html")

NameError: name 'AppTimeSeries' is not defined

# Load entities into python objects
### Using named tuple

In [ ]:
import dlite
from collections import namedtuple
import json

In [ ]:
PATH_ENTITY = "./BatteryCellMetadata.json"

In [ ]:
def custom_json_decoder(json_dict):
    return namedtuple('X', json_dict.keys())(*json_dict.values())

with open(PATH_ENTITY) as f:
    # Parse JSON into an object with attributes corresponding to dict keys.
    # json_obj = json.load(f, object_hook=custom_json_decoder)
    json_dict = json.load(f)
    json_obj = namedtuple('X', json_dict.keys())(*json_dict.values())


In [ ]:
json_obj.name

'BatteryCellMetadata'